<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/high_level_postprocessing/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import numpy as np

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Activation
from keras.callbacks import EarlyStopping

from sklearn.utils import compute_class_weight
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('local_modules')

#import postprocessing_sliding
#import postprocessing_Silvia
#import plotting

import postprocessing_sw_HL

import more_itertools

Using TensorFlow backend.


In [0]:
#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
'''
@Description: generate a multilayer perceptron with LeakyRelu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 1
#REGOLA: input/2, input, 2*input, 1
#layers TUTTE LE FEATURE: 57, 113, 226, 1 
#layers TIME CO2 TEMP: 21, 41, 82, 1 
#layers TIME CO2 TEMP PM25/TVOC: 30, 59, 118, 1 
#layers TIME CO2 TEMP PM25 TVOC: 39, 77, 154, 1 


def generate_model_leaky(shape, n_features):

  units_1 = int(n_features/2)
  units_2 = n_features
  units_3 = n_features*2

  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [0]:
'''
@Description: generate a multilayer perceptron with Relu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 2
#REGOLA: a= input, b= a*2/3+c, c= b*2/3+1
#layers TUTTE LE FEATURE: 113, 229, 153, 1
#layers TIME CO2 TEMP: 41, 85, 57, 1 
#layers TIME CO2 TEMP PM25/TVOC: 59, 121, 81, 1 
#layers TIME CO2 TEMP PM25 TVOC: 77, 157, 105, 1 

def generate_model(shape, n_features):
  
  a = np.array([[1,0,0],[-(2/3),1,-1],[0,-(2/3),1]])
  b = np.array([n_features,0,1])
  x = np.linalg.solve(a, b)

  units_1 = int(x[0])
  units_2 = int(x[1])
  units_3 = int(x[2])

  model = Sequential()
 
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [5]:
#@title SCEGLI IL DATASET E IL MODELLO

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
#print(s)
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

#print(s-1)

n_features = s-1

#SPLIT INTO TRAINING, VALIDATION AND TEST SETS
print("\nSplit into training, validation and test sets...")

train_rate = 80
val_rate = 10
train = round(int((dataset.shape[0]*train_rate)/100))
val = round(int((dataset.shape[0]*(train_rate+val_rate))/100))

train_data = X[:train]
train_label = Y[:train]

val_data = X[train+1:val]
val_label = Y[train+1:val]

test_data = X[val+1:]
test_label = Y[val+1:]
print("DATASET SPLITTED")

#COMPUTE CLASS WEIGHT
labels = np.unique(train_label)
classWeight = compute_class_weight('balanced', labels, train_label)
classWeight = dict(zip(labels,classWeight))

#GENERATE MODEL
print("\nGenerate model...")

modello = 1 #@param {type:"integer"}

if modello==1 :
  model = generate_model_leaky(train_data.shape[-1], n_features)
elif modello==2:
  model = generate_model(train_data.shape[-1], n_features)

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001)

#COMPILE MODEL
print("\nCompile model...")
model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])

#EARLY STOPPING
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

#FIT MODEL
print("\nFit model...")
history = model.fit(train_data, train_label, epochs=10, validation_data = (val_data, val_label), batch_size = 128, shuffle = True, class_weight = classWeight, verbose=1, callbacks = [es])

#EVALUATE MODEL
print("\nEvaluate model...")
scores_test = model.evaluate(test_data, test_label, batch_size=128, verbose = 1)
print("Test loss: %.2f%%" % (scores_test[0] * 100))
print("Test accuracy: %.2f%%" % (scores_test[1] * 100))

#CALCULATE PREDICTIONS
print("\nCalculate predictions...")
pred = model.predict_classes(test_data, batch_size=128, verbose=0)
flat_pred = [item for sublist in pred for item in sublist]

#CONFUSION MATRIX
print("\nCompute confusion matrix...")
y_true = test_label
y_pred = pred
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)

time = []
for i in test_data:
  time.append(i[-5])

Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE

Split into training, validation and test sets...
DATASET SPLITTED

Generate model...
Instructions for updating:
Colocations handled automatically by placer.

Compile model...

Fit model...
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 280392 samples, validate on 35048 samples
Epoch 1/10
280392/280392 [==============================] - 18s 65us/step - loss: 0.5406 - acc: 0.7603 - val_loss: 0.4591 - val_acc: 0.8477
Epoch 2/10
280392/280392 [==============================] - 17s 61us/step - loss: 0.4442 - acc: 0.8306 - val_loss: 0.4187 - val_acc: 0.8468
Epoch 3/10
280392/280392 [==============================] - 17s 62us/step - loss: 0.4135 - acc: 0.8316 - val_loss: 0.3720 - val_acc: 0.8499
Epoch 4/10
280392/280392 [==============================] - 18s 64us/step - loss: 0.3996 - acc: 0.8316 - va

In [6]:
#@title IMPOSTA LIMITI DELLA FINESTRA DI: COLAZIONE, PRANZO E CENA 
lim_inf_colazione = 330 #@param {type:"integer"}
lim_sup_colazione = 630 #@param {type:"integer"}
lim_inf_pranzo = 720 #@param {type:"integer"}
lim_sup_pranzo = 900 #@param {type:"integer"}
lim_inf_cena = 1111 #@param {type:"integer"}
lim_sup_cena = 1439 #@param {type:"integer"}

postprocessing_sw_HL.confusion_matrix_activity_pre_sw(test_label, pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena)

"""
label_time = (test_label,time)
pred_time = (pred,time)

for j in range(0,3):
  if j == 0: 
    lim_inf = lim_inf_colazione
    lim_sup = lim_sup_colazione
    activity = 'colazione'
  if j == 1: 
    lim_inf = lim_inf_pranzo
    lim_sup = lim_sup_pranzo
    activity = 'pranzo'
  if j == 2: 
    lim_inf = lim_inf_cena
    lim_sup = lim_sup_cena
    activity = 'cena'
    
  #prendo le label che ci indicano l'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup 
  real= []

  i = 0 
  while i < len(label_time[0]):
    if label_time[1][i]> lim_inf and label_time[1][i]<lim_sup:
      real.append(label_time[0][i])
    i = i+1

  #prendo le predizioni che ci indicano l'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup
  pred_sintesi = []

  i = 0 
  while i < len(pred_time[0]):
    if pred_time[1][i]>lim_inf and pred_time[1][i]<lim_sup:
      pred_sintesi.append(pred_time[0][i])
    i = i+1

  #calcolo la matrice di confusione solo per il pasto indicato dalla variabile activity
  print("\nCompute confusion matrix senza sw " + activity)
  y_true = real
  y_true = np.array(y_true,dtype=np.int)
  n_y_pred = pred_sintesi
  tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
  print("TN", tn)
  print("FP", fp)
  print("FN", fn)
  print("TP", tp)
  other = 100*tn/(tn+fp)
  pasto = 100*tp/(fn+tp)
  print("Other corretti: %.2f %%" % other)
  print("Pasto corretti: %.2f %%" % pasto)
"""


Compute confusion matrix senza sw colazione
TN 5452
FP 1437
FN 38
TP 337
Other corretti: 79.14 %
Pasto corretti: 89.87 %

Compute confusion matrix senza sw pranzo
TN 2436
FP 1259
FN 93
TP 671
Other corretti: 65.93 %
Pasto corretti: 87.83 %

Compute confusion matrix senza sw cena
TN 4786
FP 2332
FN 82
TP 632
Other corretti: 67.24 %
Pasto corretti: 88.52 %


'\nlabel_time = (test_label,time)\npred_time = (pred,time)\n\nfor j in range(0,3):\n  if j == 0: \n    lim_inf = lim_inf_colazione\n    lim_sup = lim_sup_colazione\n    activity = \'colazione\'\n  if j == 1: \n    lim_inf = lim_inf_pranzo\n    lim_sup = lim_sup_pranzo\n    activity = \'pranzo\'\n  if j == 2: \n    lim_inf = lim_inf_cena\n    lim_sup = lim_sup_cena\n    activity = \'cena\'\n    \n  #prendo le label che ci indicano l\'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup \n  real= []\n\n  i = 0 \n  while i < len(label_time[0]):\n    if label_time[1][i]> lim_inf and label_time[1][i]<lim_sup:\n      real.append(label_time[0][i])\n    i = i+1\n\n  #prendo le predizioni che ci indicano l\'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup\n  pred_sintesi = []\n\n  i = 0 \n  while i < len(pred_time[0]):\n    if pred_time[1][i]>lim_inf and pred_time[1][i]<lim_sup:\n      pred_sintesi.append(pred_time[0][i])\n    i = i+1\n\n  #calc

In [7]:
#@title IMPOSTA LA GRANDEZZA DELLA FINESTRA DI: COLAZIONE, PRANZO E CENA 
finestra_colazione = 5 #@param {type:"integer"}
finestra_pranzo = 31 #@param {type:"integer"}
finestra_cena = 31 #@param {type:"integer"}
min_pasti_colazione = 3 #@param {type:"integer"}
min_pasti_pranzo = 15 #@param {type:"integer"}
min_pasti_cena = 15 #@param {type:"integer"}

sw_real_activity = postprocessing_sw_HL.sw_for_real_activity(test_label,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena,finestra_colazione,finestra_pranzo,finestra_cena)
sw_col_real = sw_real_activity[0]
sw_pranzo_real = sw_real_activity[1]
sw_cena_real = sw_real_activity[2]

"""
#calcolo le sliding windows per ogni tipologia di pasto (colazione, pranzo, cena) nell'array in cui sono presenti le attività reali 
#per ogni sliding windows mi salvo il minuto centrale come identificatore dell'attività che si è realmente svolta in quella finestra di tempo

sw_col_real = []
sw_pranzo_real = []
sw_cena_real = []

for j in range(0,3):
  if j == 0: 
    lim_inf = lim_inf_colazione
    lim_sup = lim_sup_colazione
    finestra = finestra_colazione
  if j == 1: 
    lim_inf = lim_inf_pranzo
    lim_sup = lim_sup_pranzo
    finestra = finestra_pranzo
  if j == 2: 
    lim_inf = lim_inf_cena
    lim_sup = lim_sup_cena
    finestra = finestra_cena
    
  #estrazione sliding windows 
  pasto = list(more_itertools.windowed(test_label, n=finestra, step=1))
  time_pasto = list(more_itertools.windowed(time, n=finestra, step=1))
  lista = (pasto,time_pasto)
  
  sw_real = []
  
  i = 0 
  while i < len(lista[0]):
    if lista[1][i][0]>lim_inf and lista[1][i][finestra-1]<lim_sup:
      centro = int((finestra-1)/2)
      sw_real.append(lista[0][i][centro])
    i = i+1
    
  if j == 0:
    sw_col_real = sw_real
  if j == 1:
    sw_pranzo_real = sw_real
  if j == 2:
    sw_cena_real = sw_real
    
#print(sw_col_real)
#print(len(sw_col_real))
#print(sw_pranzo_real)
#print(len(sw_pranzo_real))
#print(sw_cena_real)
#print(len(sw_cena_real))
"""

"\n#calcolo le sliding windows per ogni tipologia di pasto (colazione, pranzo, cena) nell'array in cui sono presenti le attività reali \n#per ogni sliding windows mi salvo il minuto centrale come identificatore dell'attività che si è realmente svolta in quella finestra di tempo\n\nsw_col_real = []\nsw_pranzo_real = []\nsw_cena_real = []\n\nfor j in range(0,3):\n  if j == 0: \n    lim_inf = lim_inf_colazione\n    lim_sup = lim_sup_colazione\n    finestra = finestra_colazione\n  if j == 1: \n    lim_inf = lim_inf_pranzo\n    lim_sup = lim_sup_pranzo\n    finestra = finestra_pranzo\n  if j == 2: \n    lim_inf = lim_inf_cena\n    lim_sup = lim_sup_cena\n    finestra = finestra_cena\n    \n  #estrazione sliding windows \n  pasto = list(more_itertools.windowed(test_label, n=finestra, step=1))\n  time_pasto = list(more_itertools.windowed(time, n=finestra, step=1))\n  lista = (pasto,time_pasto)\n  \n  sw_real = []\n  \n  i = 0 \n  while i < len(lista[0]):\n    if lista[1][i][0]>lim_inf and lis

In [8]:
sw_pred_activity = postprocessing_sw_HL.sw_for_pred_activity(pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena,lim_sup_cena,finestra_colazione,finestra_pranzo,finestra_cena, min_pasti_colazione, min_pasti_pranzo, min_pasti_cena)
result_col = sw_pred_activity[0]
result_pranzo = sw_pred_activity[1]
result_cena = sw_pred_activity[2]

"""
#calcolo le sliding windows per ogni tipologia di pasto (colazione, pranzo, cena) nell'array delle predizioni
sw_col = []
sw_pranzo = []
sw_cena = []

for j in range(0,3):
  if j == 0:  
    lim_inf = lim_inf_colazione
    lim_sup = lim_sup_colazione
    finestra = finestra_colazione
  if j == 1: 
    lim_inf = lim_inf_pranzo
    lim_sup = lim_sup_pranzo
    finestra = finestra_pranzo
  if j == 2: 
    lim_inf = lim_inf_cena
    lim_sup = lim_sup_cena
    finestra = finestra_cena
  
  #estrazione sliding windows
  pred_sws = list(more_itertools.windowed(pred, n=finestra, step=1))
  time_sws = list(more_itertools.windowed(time, n=finestra, step=1))
  lista = (pred_sws,time_sws)
  
  sw = []
  
  i = 0 
  while i < len(lista[0]):
    if lista[1][i][0]>lim_inf and lista[1][i][finestra-1]<lim_sup:
      sw.append(lista[0][i])
    i = i+1
  
  print(len(sw))
  if j == 0:
    sw_col = sw    
  if j == 1:
    sw_pranzo = sw
  if j == 2:
    sw_cena = sw

#creazione di tre array in cui si inserisce il minuto centrale di una sw se questa ci sembra una colazione, un pranzo o una cena    
result_col = [];
result_pranzo = [];
result_cena = [];

for j in range(0,3):
  if j == 0: 
    windowsList = sw_col
    min_pasto = min_pasti_colazione
  if j == 1: 
    windowsList = sw_pranzo
    min_pasto = min_pasti_pranzo
  if j == 2: 
    windowsList = sw_cena
    min_pasto = min_pasti_cena
  
  result = []
  i = 0 
  
  while i < len(windowsList): 
    count = 0

    z = 0  
    while z < len(windowsList[i]):
      if windowsList[i][z] == 1 :
        count = count + 1
      z = z + 1

    if count > min_pasto:
      result.append(1)
    else:
      result.append(0)

    i = i + 1
  
  if j == 0:
    result_col = result
  if j == 1:
    result_pranzo = result
  if j == 2:
    result_cena = result

#print(result_col)
#print(len(result_col))
#print(result_pranzo)
#print(len(result_pranzo))
#print(result_cena)
#print(len(result_cena))
"""

7260
4429
7802


"\n#calcolo le sliding windows per ogni tipologia di pasto (colazione, pranzo, cena) nell'array delle predizioni\nsw_col = []\nsw_pranzo = []\nsw_cena = []\n\nfor j in range(0,3):\n  if j == 0:  \n    lim_inf = lim_inf_colazione\n    lim_sup = lim_sup_colazione\n    finestra = finestra_colazione\n  if j == 1: \n    lim_inf = lim_inf_pranzo\n    lim_sup = lim_sup_pranzo\n    finestra = finestra_pranzo\n  if j == 2: \n    lim_inf = lim_inf_cena\n    lim_sup = lim_sup_cena\n    finestra = finestra_cena\n  \n  #estrazione sliding windows\n  pred_sws = list(more_itertools.windowed(pred, n=finestra, step=1))\n  time_sws = list(more_itertools.windowed(time, n=finestra, step=1))\n  lista = (pred_sws,time_sws)\n  \n  sw = []\n  \n  i = 0 \n  while i < len(lista[0]):\n    if lista[1][i][0]>lim_inf and lista[1][i][finestra-1]<lim_sup:\n      sw.append(lista[0][i])\n    i = i+1\n  \n  print(len(sw))\n  if j == 0:\n    sw_col = sw    \n  if j == 1:\n    sw_pranzo = sw\n  if j == 2:\n    sw_cena = s

In [9]:
postprocessing_sw_HL.confusion_matrix_activity_post_sw(sw_col_real, sw_pranzo_real, sw_cena_real,result_col, result_pranzo, result_cena)

"""
print("\nCompute confusion matrix colazione...")
y_true = sw_col_real
y_true = np.array(y_true,dtype=np.int)
n_y_pred = result_col
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)

print("\nCompute NEW confusion matrix pranzo...")
y_true = sw_pranzo_real
y_true = np.array(y_true,dtype=np.int)
n_y_pred = result_pranzo
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)

print("\nCompute NEW confusion matrix cena...")
y_true = sw_cena_real
y_true = np.array(y_true,dtype=np.int)
n_y_pred = result_cena
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
"""


Compute confusion matrix colazione...
TN 5597
FP 1290
FN 50
TP 323
Other corretti: 81.27 %
Pasto corretti: 86.60 %

Compute NEW confusion matrix pranzo...
TN 2436
FP 1230
FN 69
TP 694
Other corretti: 66.45 %
Pasto corretti: 90.96 %

Compute NEW confusion matrix cena...
TN 4897
FP 2193
FN 81
TP 631
Other corretti: 69.07 %
Pasto corretti: 88.62 %


'\nprint("\nCompute confusion matrix colazione...")\ny_true = sw_col_real\ny_true = np.array(y_true,dtype=np.int)\nn_y_pred = result_col\ntn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()\nprint("TN", tn)\nprint("FP", fp)\nprint("FN", fn)\nprint("TP", tp)\nother = 100*tn/(tn+fp)\npasto = 100*tp/(fn+tp)\nprint("Other corretti: %.2f %%" % other)\nprint("Pasto corretti: %.2f %%" % pasto)\n\nprint("\nCompute NEW confusion matrix pranzo...")\ny_true = sw_pranzo_real\ny_true = np.array(y_true,dtype=np.int)\nn_y_pred = result_pranzo\ntn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()\nprint("TN", tn)\nprint("FP", fp)\nprint("FN", fn)\nprint("TP", tp)\nother = 100*tn/(tn+fp)\npasto = 100*tp/(fn+tp)\nprint("Other corretti: %.2f %%" % other)\nprint("Pasto corretti: %.2f %%" % pasto)\n\nprint("\nCompute NEW confusion matrix cena...")\ny_true = sw_cena_real\ny_true = np.array(y_true,dtype=np.int)\nn_y_pred = result_cena\ntn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel(

In [10]:
#@title SCEGLI maxWidth per postprocessing_Silvia
'''
@Description: POST PROCESSING
'''

#maxWidth = 3 #@param {type:"integer"}

#new_pred = postprocessing_sliding.sliding_windows_time(flat_pred, time)
#new_pred = postprocessing_sliding.sliding_windows_time_mod(flat_pred, time)
#new_pred = postprocessing_Silvia.setWidth(maxWidth,flat_pred)

#CONFUSION MATRIX
"""
print("\nCompute NEW confusion matrix...")
y_true = test_label
n_y_pred = new_pred
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)
"""

'\nprint("\nCompute NEW confusion matrix...")\ny_true = test_label\nn_y_pred = new_pred\ntn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()\nprint("TN", tn)\nprint("FP", fp)\nprint("FN", fn)\nprint("TP", tp)\nother = 100*tn/(tn+fp)\npasto = 100*tp/(fn+tp)\nprint("Other corretti: %.2f %%" % other)\nprint("Pasto corretti: %.2f %%" % pasto)\n'

In [11]:
'''
@Description: PLOTTING
'''
'''
plotting.plot_model_results(history)
plotting.plot_co2_temp(flat_pred, val)
'''

'\nplotting.plot_model_results(history)\nplotting.plot_co2_temp(flat_pred, val)\n'

In [0]:
#model.save('my_model1.h5')
#model = load_model('my_model.h5')